# jieba pathon API

In [ ]:
import jieba

In [ ]:
# 搜索引擎模式
seg_list = jieba.cut_for_search("小明硕士毕业于中国科学院计算所，后在日本京都大学深造")  

In [ ]:
dict(seg_list)

In [ ]:
chilist = list(seg_list)

In [ ]:
chilist

# 加载文件

In [ ]:
import jieba
from pysqlite2 import dbapi2 as sqlite
import re
import math
import os
import string

In [ ]:
# 搜索引擎模式
def separatewords(text):
    # 搜索引擎模式
    seg_list = jieba.cut_for_search(text)  
    words = [s for s in list(seg_list) if len(s)>2 and len(s) < 20]
    return words

In [ ]:
# 遍历所有文件，提取标题和正本
def getarticlewords(rootdir):
    # 默认的 0 值，为了防止除 0 
    mincount = 0.000000001
    # 所有单词计数
    allwords = {}
    # 文章单词计数
    articlewords = []
    # 文章标题
    articletitles = []
    # 文章数组索引值
    ec = 0
    
    # Loop over every file
    i = 0
    for parent,dirnames,filenames in os.walk(rootdir):
        #输出文件信息
        for filename in filenames:                        
            #print "parent is:" + parent
            #print type(parent)
            #print "filename is: " + str(filename).decode('gbk', 'ignore')
            #输出文件路径信息
            #print "the full name of the file is:" + os.path.join(parent,filename) 
            file_class = parent.split("\\")[1]
            #print file_class
            full_name = os.path.join(parent,filename) 
            i += 1
            print str(i) + " ,full_name is " + str(full_name).decode('gbk', 'ignore')
            # 读取文件
            title,author,zhan_num,url,content = read_and_parse_file(full_name)
            if title == None:
                continue
                
            # Extract the words
            txt = title + " " + content
            words = separatewords(txt)
            articlewords.append({})
            # 文章标题
            articletitles.append(title)

            # Increase the counts for this word in allwords and in articlewords
            for word in words:
                allwords.setdefault(word,mincount)
                # 该单词在所有文章中的总计数
                allwords[word]+=1
                articlewords[ec].setdefault(word,mincount)
                # 该单词在该文章中的单词计数
                articlewords[ec][word]+=1
            #
            ec+=1
                  
    # 返回 单词频次，文章中单词频次，文章标题
    return allwords,articlewords,articletitles

In [ ]:
def read_and_parse_file(file_path):
    # 去掉标点符号
    delset = string.punctuation
    try:
        if os.path.exists(file_path):
            f = open(file_path, "r")
            lines = f.readlines()
            f.close()
            if len(lines) < 5:
                return None,None,0,None,None
            title = lines[0].strip()
            fields = lines[2].split(':')
            zhan_num = int(fields[2].strip())
            author = fields[1].strip().split(' ')[0].strip()
            content = ''.join(lines[5:-2])
            url = lines[-1].split(' ')[1].strip()
            #
            title = title.translate(None,delset)
            author = author.translate(None,delset)
            content = content.translate(None,delset)
            return title,author,zhan_num,url,content
    except IOError:
        error_file_path = file_path
        print "error_file_path:" , error_file_path
    
    # 默认
    return None,None,0,None,None

# 转换成矩阵

In [ ]:
# 【单词频次 , 文章中单词频次】
def makematrix(allw,articlew):
    # 默认的 0 值，为了防止除 0 
    mincount = 0.000000001

    wordvec = []

    # Only take words that are common but not too common
    # 过滤掉单词太频繁和太不频繁的单词
    for w,c in allw.items():
        # 至少出现过一次并且最多在 60% 的文章中出现过，否则太频繁了
        if c>3 and c<len(articlew)*0.6:
            wordvec.append(w) 

    # Create the word matrix
    #l1=[[(word in f and f[word] or 0) for word in wordvec] for f in articlew]
    # 矩阵
    # 行 ： 文章 
    # 列 ： 单词
    # 值 ： 单词在文章中出现的次数
    l1=[[(word in f and f[word] or mincount) for word in wordvec] for f in articlew]

    # 返回 文章单词矩阵, 单词列表(矩阵的列)
    return l1,wordvec

In [ ]:
rootdir = 'E:/project/pychram/zhihu/zhihu-python/text'

In [ ]:
# 单词频次，文章中单词频次，文章标题
allwords,articlewords,articletitles = getarticlewords(rootdir)


In [ ]:

# 文章单词矩阵, 单词列表(矩阵的列)
wordmatrix,wordvec = makematrix(allwords,articlewords)



In [ ]:
wordvec[0:10]


In [ ]:
articletitles[1]


In [ ]:
wordmatrix[1][0:10]


In [ ]:
len(allwords)

# 因式分解函数

In [ ]:
from numpy import *

# 计算两个矩阵的距离差平方累加值
def difcost(a,b):
    dif=0
    for i in range(shape(a)[0]):
        for j in range(shape(a)[1]):
            # Euclidean Distance
            dif+=pow(a[i,j]-b[i,j],2)
    return dif

# 分解矩阵为 权重矩阵 * 特征矩阵
# 【输入矩阵,需要输出的特征数量,迭代次数】
def factorize(v,pc=10,iter=50):
    ic=shape(v)[0]
    fc=shape(v)[1]

    # Initialize the weight and feature matrices with random values
    w=matrix([[random.random() for j in range(pc)] for i in range(ic)])
    h=matrix([[random.random() for i in range(fc)] for i in range(pc)])

    # Perform operation a maximum of iter times
    for i in range(iter):
        wh=w*h

        # Calculate the current difference
        cost=difcost(v,wh)

        if i%10==0: print cost

        # Terminate if the matrix has been fully factorized
        if cost==0: break

        # Update feature matrix
        hn=(transpose(w)*v)
        hd=(transpose(w)*w*h)

        h=matrix(array(h)*array(hn)/array(hd))

        # Update weights matrix
        wn=(v*transpose(h))
        wd=(w*h*transpose(h))

        w=matrix(array(w)*array(wn)/array(wd))  

    return w,h


In [ ]:
v = matrix(wordmatrix)


In [ ]:
weights,feat = factorize(v,pc=10,iter=20)

In [ ]:
# 遍历特征矩阵,显示每一行特征的 前 6 个特征次数最多的单词
# 遍历权重矩阵,显示该特征(列) 的行(文章) 的权重值最高的 3 篇文章标题
def showfeatures(w,h,titles,wordvec,out='features.txt'): 
    outfile = file(out,'w')  
    # pc 行：特征
    # wc 列：单词
    # [pc,wc] : 单词在特征中出现的次数
    pc,wc = shape(h)
    # 行： 标题文章
    toppatterns = [[] for i in range(len(titles))]
    patternnames = []

    # Loop over all the features
    # 遍历每一行：特征
    pos = 0
    for i in range(pc):
        pos += 1
        slist = []
        # Create a list of words and their weights
        # 遍历每一列 ：单词
        for j in range(wc):
            # [pc,wc] : 单词在特征中出现的次数 ，单词的内容
            slist.append((h[i,j],wordvec[j]))
        # Reverse sort the word list
        # 按 特征次数排序，降序
        slist.sort()
        slist.reverse()

        # Print the first six elements
        # 这一特征行中 前 6 位的 单词的内容数组
        n = [s[1] for s in slist[0:6]]
        # 这一特征行中 显示 前 6 个单词内容
        n_str_list = [ item.encode('UTF-8','ignore') for item in n]
        n_str = ','.join(n_str_list)
        outfile.write(n_str + '\n')
        print str(pos) + " : " + n_str + '\n'
        
        # 将 前 6 位的 单词的内容数组 放入模式名称数组中
        # 行 ： 特征
        # 列 ： 前 6 位的 单词
        patternnames.append(n)

        # Create a list of articles for this feature
        
        flist=[]
        # 遍历每一篇文章标题
        # 行：文章
        for j in range(len(titles)):
            # Add the article with its weight
            # 列 ： 特征
            # [文章,特征] ： 特征在文章中的权重，文章标题
            flist.append((w[j,i],titles[j]))
            # 行：文章 
            # 列：(特征权重,特征矩阵中的特征索引-行索引,文章标题)
            toppatterns[j].append((w[j,i],i,titles[j]))

        # Reverse sort the list
        # 按 特征在文章中的权重 排序，倒序
        flist.sort()
        flist.reverse()

        # Show the top 3 articles
        # 这一行特征中，该特征权重最大的 3 篇文章
        for f in flist[0:3]:
            # 特征在文章中的权重,文章标题
            # 显示需要对字符串解码为 GBK
            print f[0], f[1].decode('gbk','ignore') + '\n'
            
            n_str = str(f[0]) + ", " +  (f[1].decode('gbk','ignore')) + '\n'
            #print n_str
            
            # 写入文件需要对字符串加码为 UTF-8
            outfile.write(n_str.encode('UTF-8','ignore') + '\n')
            
        outfile.write('\n')

    outfile.close()
    # Return the pattern names for later use
    # 返回 
    # toppatterns ： 
            # 行：文章 
            # 列：(特征权重,特征索引,文章标题)
    # patternnames
            # 行 ： 特征
            # 列 ： 前 6 位的 单词
    return toppatterns,patternnames


In [ ]:

topp,pn = showfeatures(weights,feat,articletitles,wordvec)




In [ ]:
# 显示文章及权重最大的特征的前6个单词
# 【# titles: 文章标题
    # toppatterns ： 
            # 行：文章 
            # 列：(特征权重,特征矩阵中的特征索引-行索引,文章标题)
    # patternnames
            # 行 ： 特征
            # 列 ： 前 6 位的 单词】
def showarticles(titles,toppatterns,patternnames,out='articles.txt'):
    outfile = file(out,'w')  

    # Loop over all the articles
    # 遍历每一篇文章
    pos = 0
    for j in range(len(titles)):
        pos += 1
        # 打印这篇文章的标题
        title_str = str(pos) + " : " + titles[j].decode('gbk','ignore') + '\n'
        print title_str
        #
        outfile.write(title_str.encode('UTF-8','ignore'))

        # Get the top features for this article and
        # reverse sort them
        # 这篇文章的特征权重排序后，降序
        toppatterns[j].sort()
        toppatterns[j].reverse()

        # Print the top three patterns
        # 显示这篇文章的最大权重的前 3 个特征
        # 遍历这篇文章的前3个特征
        for i in range(3):
            # 显示该特征在该文章中的权重
            # 显示该特征在给文章中的前 6 位的特征单词
            fea_str = str(toppatterns[j][i][0]) + " : " + ','.join(patternnames[toppatterns[j][i][1]]) + '\n'
            print fea_str
            #
            outfile.write(fea_str.encode('UTF-8','ignore') )
        outfile.write('\n')

    outfile.close()

showarticles(articletitles,topp,pn)